In [103]:
using LinearAlgebra


include("Algoritmo LU.jl")

LUPP (generic function with 1 method)

# Método de Newon para cálculo de valores y vectores propios.

Resuelva el ejercicio 5.29 de la página 254 del libro Scientific computing, an introdutory survey de M. Heath.


Tomamos una matriz $A$ de $n \times n$. Definimos la función $f \colon \mathbb{R}^{n+1} \to \mathbb{R}^{n+1}$ como
    \begin{align}
        f(x,\lambda) = \begin{bmatrix} Ax - \lambda x \\ x^T x-1 \end{bmatrix}
    \end{align}
Entonces el método de Newton para resolver la ecuación $f(x,\lambda)=0$ toma la forma
    \begin{align}
        \begin{bmatrix} 
            x_{k+1} \\
            \lambda_{k+1}
        \end{bmatrix}
        = \begin{bmatrix}
            x_k \\ \lambda_k
        \end{bmatrix} + \begin{bmatrix}
            s_k \\ \delta_k
        \end{bmatrix},
    \end{align}
donde $\begin{bmatrix} s_k & \delta_k \end{bmatrix}^T$ es la solución del sistema de ecuaciones
    \begin{align}
        \begin{bmatrix}
            A - \lambda_k I & -x_k \\
            2 x_k^T & 0
        \end{bmatrix}
        \begin{bmatrix}
            s_k \\ \delta_k
        \end{bmatrix}
        & = - \begin{bmatrix}
            Ax_k - \lambda_k x_k \\
            x_k^T x_k -1
        \end{bmatrix}
    \end{align}

In [363]:
# Implementación del método de Newton

# Usaremos el método de Householder para calcular la factorización QR del jacobiano

#Esta función toma como argumento una matriz A y regresa la matriz de Householder H tal que
# HA es una matriz cuya primera columna tiene puros ceros debajo de la diagonal
function Householder(A)
    x = A[:,1]

    e1 = zeros(size(A)[1])
    e1[1] = 1.0

    v = x+sign(A[1,1])*norm(x)*e1
    v = v/norm(v)

    H = I -2*v*v'
end

#Esta función toma como argumento una matriz A de mxn. Regresa una tupla (Q,R) donde Q es una matriz ortogonal y
# R es una matriz triangular superior tales que QR = A.
function QRHouseholder(A)
    (m,n) = size(A)
    Q = Matrix(1.0*I,size(A)[1],size(A)[1])
    R = A

    for i = 1:size(A)[2]
        HTild = Householder(R[i:size(A)[1],i:size(A)[2]] )
    
        H = Matrix(1.0*I,size(A)[1],size(A)[1])
        H[i:size(A)[1],i:size(A)[1]] = HTild
    
        R = H*R
        Q = H*Q
    end
    Q = Q'
    return(Q,R)
end

# Tomamos como input una matriz A de nxn



# Definimos la función f. x debe ser un vector de nx1 y y debe ser un número real
function f(x,y)
    n = length(x)
    B = zeros(n+1)
    B[1:n] = A*x-y*x
    B[n+1] = x'*x - 1
    return(B)
end


A = [1 0 0; 0 5 0; 0 0 9]
(n,m) = size(A)

# Aproximación inicial
x0 = [1; -1; 1]
x0 = x0/norm(x0)

lambda0 = (x0)'*A*(x0)

# Máximo de iteraciones
nit = 1000

# Tolerancia. El algortimo se detiene cuando norm( f( x_k, lambda_k) < tol
tol = 10.0^(-16)

xk = x0
lambdak = lambda0

C = zeros(n+1,n+1)
c = zeros(n+1)

for i = 1:nit
    if norm( f(xk,lambdak) ) <= tol
        println(i)
        break
    else
        # Consideramos el siguiente sistema de ecuacions C [sk;deltak] = c
        C[1:n,1:n] = A-lambdak*I
        C[1:n,n+1] = -(xk)
        C[n+1,1:n] = 2*(xk)'
    
        c[1:n] = -(A*xk - lambdak*xk )
        c[n+1] = -( ( (xk)'*xk )-1 )
    
        #Calculamos la factorización QR de C
        (Q,R) = QRHouseholder(C)
    
        #Resolvemos el sistema de (n+1) x (n+1):  R [s, delta] = Q'*c por sustitución hacia atrás
        SDelta = SolBwd(R, Q'*c)
    
        xk = xk + SDelta[1:n]
        # Normalizamos a xk
        xk = xk/norm(xk)
        lambdak = lambdak + SDelta[n+1]
    end        
end


3


In [269]:
# Implementación del método de la potencia

# Tomamos como input una matriz A de nxn y una aproximación inicial no nula
A = [1 0 0; 0 5 0; 0 0 9]
x0 = [1; -1; 1]



3-element Vector{Int64}:
  1
 -1
  1

In [362]:
# Implementación del método de la potencia normalizada 

# Tomamos como input una matriz A de nxn y una aproximación inicial no nula
A = [1 0 0; 0 5 0; 0 0 9]
x0 = [1; -1; 1]
x0 = x0/norm(x0)
# Máximo número de iteraciones y tolerancia
nit = 1000000
tol = 10.0^(-16)
xk = x0
lambdak = 0

for i = 1:nit
    if norm((A-lambdak*I)*xk)<= tol
        print(i)
        break
    else
        # Este índice es para encontrar la aproximación actual de lambda. Se define
        # k0 de esta forma para evitar tomar una entrada de xk igual a 0 (o muy cercana a 0)
        k0 = findfirst(a -> a == maximum( abs.(xk) ), abs.(xk))
        lambdak = (A*xk)[k0]/xk[k0]
        xk = A*xk  
        xk = xk/norm(xk)
    end
end


67

In [360]:
lambdak


9.0

In [352]:
lambda = ((A*xk)[1])/(xk[1])

1.0

In [337]:
A

3×3 Matrix{Int64}:
 1  0  0
 0  5  0
 0  0  9

In [350]:
xk

3-element Vector{Float64}:
  1.0471149905198215e-63
 -1.4191054344546608e-17
  1.0

In [351]:
A*xk

3-element Vector{Float64}:
  1.0471149905198215e-63
 -7.095527172273304e-17
  9.0

In [246]:
A*xk


3-element Vector{Float64}:
  5.127900497022836e-16
 -5.0
 -4.615110447320552e-15

In [247]:
lambdak*xk


3-element Vector{Float64}:
  2.5639502485114157e-15
 -4.999999999999996
 -2.5639502485114157e-15